# Biospecimen data preparation
This script prepares and combines available biospecimen markers data from multiple sources into a single file, and also applies any needed preprocessing steps to the various types of data
Output should be a file named biospecimen_data.csv with:

| RID | DX | Gender | Age | Ed. level | all the other markers... |


In [9]:
# do imports
import pandas as pd
import numpy as np

**Load all relevant data tables**

First, we load all relevant tables in pandas DataFrames.

In [10]:
# ADNIMERGE includes general information about the patients
# Not sure I will need it, but whatever
df_data = pd.read_csv('ADNIMERGE.csv')
df_plasma_full = pd.read_csv('adni_plasma_qc_multiplex_11Nov2010.csv')

# NFL data
df_nfl=pd.read_csv('ADNI_BLENNOWPLASMANFL.csv')

# Plasma ABETA UPENN
df_abeta=pd.read_csv('UPENNPLASMA.csv')

# HOMOCYSTEINE
df_homocysteine=pd.read_csv('HCRES.csv')

## ADMC DATA
# P180 data
df_metabolytes=pd.read_csv('ADMCDUKEP180UPLC_01_15_16.csv')

In [11]:
# Select interesting columns
info = ["RID", "PTID", "MMSE", "VISCODE", "EXAMDATE",
        "AGE", "PTGENDER", "APOE4", "DX_bl", "PTEDUCAT"]

df_data = df_data[info].copy()

# Select only baseline data
df_data = df_data[(df_data.VISCODE == "bl")]

# Drop -1 and -4 missing indicators
df_data = df_data.replace(to_replace=[-1, -4], value=[np.nan, np.nan]).dropna()

# ADD Plasma biomarkers
# Drop empty columns
df_plasma_full = df_plasma_full.replace(to_replace=['.'], value=[np.nan])
df_plasma_full = df_plasma_full.dropna(axis=1, how='any')

# select only baselines
df_plasma_full = df_plasma_full[df_plasma_full.Visit_Code == 'bl']

# Add all colums
df_data = pd.merge(df_data, df_plasma_full.iloc[:, 1:], how='inner', on="RID")
del df_data['Visit_Code']
del df_data['RBM Sample ID']
del df_data['Sample_Received_Date']

# Drop missing data, print some statistics of the data
df_data.dropna()

,RID,PTID,MMSE,VISCODE,EXAMDATE,AGE,PTGENDER,APOE4,DX_bl,PTEDUCAT,...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,3,011_S_0003,20.0,bl,2005-09-12,81.3,Male,1.0,AD,18,...,1.30,1.34242,3.41330,0.55630,2.68305,2.98722,2.97035,2.96379,1.23045,1.14613
1,4,022_S_0004,27.0,bl,2005-11-08,67.5,Male,0.0,LMCI,10,...,2.20,1.39794,3.34439,0.04139,2.64542,2.84136,2.95904,3.01284,1.32222,1.89209
2,6,100_S_0006,25.0,bl,2005-11-29,80.4,Female,0.0,LMCI,13,...,2.30,1.43136,3.60959,0.46240,2.60097,3.09691,2.94448,3.01284,1.30103,1.17609
3,8,011_S_0008,28.0,bl,2005-09-19,84.5,Female,0.0,CN,18,...,2.10,0.75587,3.46982,0.32222,2.67943,2.86153,2.81954,2.88986,1.41497,1.89763
4,10,011_S_0010,24.0,bl,2005-11-10,73.9,Female,1.0,AD,12,...,1.30,1.23045,3.35984,0.41497,2.40654,2.88024,2.74741,2.88536,1.23045,1.51851
5,14,022_S_0014,29.0,bl,2005-11-04,78.5,Female,0.0,CN,12,...,1.00,1.14613,3.41830,0.07918,2.68124,2.68124,2.73400,2.90526,1.30103,1.77815
6,19,067_S_0019,29.0,bl,2005-11-23,73.1,Female,0.0,CN,18,...,2.50,1.20412,3.66464,0.07918,2.58320,3.14613,2.71684,2.92012,1.34242,1.41497
7,22,011_S_0022,29.0,bl,2005-10-19,63.2,Male,1.0,CN,17,...,0.50,0.96848,3.60853,0.46240,2.93146,2.92221,2.84819,3.16435,1.51851,1.91381
8,23,011_S_0023,26.0,bl,2005-11-08,71.7,Male,0.0,CN,14,...,1.90,1.23045,3.65031,0.30103,2.48996,2.84198,2.79449,3.04532,1.44716,1.74036
9,29,067_S_0029,21.0,bl,2005-10-31,64.1,Male,1.0,AD,18,...,0.51,1.07918,3.51055,0.43136,2.53403,2.84136,2.75358,2.88536,1.34242,1.63347


Select only the subset of the data where:
* No missing data values.
* Data quality is good across all markers

Also, we want to check whether the number of subjects is high enough for the tests, so tests different combinations

In [12]:
# First, without the metabolytes
# Keep adding the data to the origina and check how many subjects still remain

# ADD homocysteine
df_homocysteine = df_homocysteine[df_homocysteine.VISCODE == 'bl']
df_homocysteine = df_homocysteine[["RID", "HCAMPLAS"]]
print(df_homocysteine.isnull().sum())
df_homocysteine.dropna()
df_useddata_homo = pd.merge(df_data, df_homocysteine, how='inner', on="RID")

# ADD PLasma ABETA
df_abeta = df_abeta[df_abeta.VISCODE == 'bl']
df_abeta = df_abeta[["RID", "AB40", "AB42"]]
print(df_abeta.isnull().sum())
df_abeta.dropna()
df_useddata_homo_abeta = pd.merge(df_useddata_homo, df_abeta, how='inner', on="RID")

# Add NFL data
df_nfl = df_nfl[df_nfl.VISCODE == 'bl']
df_nfl = df_nfl[["RID", "PLASMA_NFL"]]
print(df_nfl.isnull().sum())
df_nfl.dropna()
df_useddata_homo_abeta_plasma = pd.merge(df_useddata_homo_abeta, df_nfl, how='inner', on="RID")

RID         0
HCAMPLAS    0
dtype: int64
RID      0
AB40    10
AB42     7
dtype: int64
RID           0
PLASMA_NFL    4
dtype: int64


In [13]:
# Cell for testing
df_useddata_homo_abeta_plasma
print('Total samples: ' + str(len(df_useddata_homo_abeta_plasma)))
print("AD samples: " + str(len(df_useddata_homo_abeta_plasma[df_useddata_homo_abeta_plasma.DX_bl == 'AD'])))
print("LMCI samples: " + str(len(df_useddata_homo_abeta_plasma[df_useddata_homo_abeta_plasma.DX_bl == 'LMCI'])))
print("CN samples: " + str(len(df_useddata_homo_abeta_plasma[df_useddata_homo_abeta_plasma.DX_bl == 'CN'])))

Total samples: 333
AD samples: 96
LMCI samples: 181
CN samples: 56


In [15]:
# Try to add the metabolytes

# Integrate columns names from dictionary
# load dictionary
df_metabolytes_DICT=pd.read_csv('ADMCDUKEP180UPLC_DICT.csv')
df_metabolytes = df_metabolytes.drop_duplicates(subset=['RID'], keep='first', inplace=False)

# Convert to NA non-numeric values

def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

df_metabolytes = df_metabolytes[df_metabolytes.applymap(isnumber)]

# drop columns that have more than 10% NA values
df_metabolytes = df_metabolytes.dropna(axis=1, how='any')
print(len(df_metabolytes))
print(df_metabolytes)
#After this, remove individual samples
df_metabolytes = df_metabolytes.dropna()
print(len(df_metabolytes))
df_useddata_homo_abeta_plasma_meta = pd.merge(df_useddata_homo_abeta_plasma, df_metabolytes, how='inner', on="RID")
# assign new names

# Check statistics
print('Total samples: ' + str(len(df_useddata_homo_abeta_plasma_meta)))
print("AD samples: " + str(len(df_useddata_homo_abeta_plasma_meta[df_useddata_homo_abeta_plasma_meta.DX_bl == 'AD'])))
print("LMCI samples: " + str(len(df_useddata_homo_abeta_plasma_meta[df_useddata_homo_abeta_plasma_meta.DX_bl == 'LMCI'])))
print("CN samples: " + str(len(df_useddata_homo_abeta_plasma_meta[df_useddata_homo_abeta_plasma_meta.DX_bl == 'CN'])))

df_useddata_homo_abeta_plasma_meta
df_useddata_homo_abeta_plasma_meta.to_csv("useddata_homo_abeta_plasma_meta.csv")

808
        RID  Alanine  Arginine  Asparagine Aspartic Acid  Citrulline  \
0         2      651     155.0        54.7          22.9        49.9   
1         3      557     134.0        74.5          19.2        49.9   
2         4      398     192.0        48.4          36.6        77.9   
3         5      599     163.0        60.6          19.2        35.8   
4         6      520     174.0        74.1          22.1        61.3   
5         7      475     148.0        58.4          31.6        54.7   
6         8      635     185.0        58.3          27.3        42.9   
7        10      370     126.0        55.4          23.3        49.3   
8        14      456     213.0        69.8          54.3        51.1   
9        15      344     174.0        47.9          27.4        43.2   
10       16      544     188.0        48.1            28        28.3   
11       19      401     181.0        60.8          26.7        62.9   
12       21      431     136.0        58.6            30    